# Phần preprocessing

Thư viện

In [15]:
import os
import numpy as np
import pandas as pd

Định nghĩa các thông số

In [16]:
step = 32 # Time_step
X = [] # Data sau cùng
Y = [] # Label (1 là tục, 0 là không tục)

Lấy data

In [17]:
list_folder_name = os.listdir('DATA')
min_line = 1e9
list_len_frame = []

for folder in list_folder_name:
    list_file_error = []
    path_folder_parent = os.path.join('DATA', folder)
    
    path_folder_tuc = os.path.join(path_folder_parent, 'DATA_TUC')
    path_folder_khong_tuc = os.path.join(path_folder_parent, 'DATA_KHONG_TUC')

    quantity_file = 0
    # Data tục
    for file in os.listdir(path_folder_tuc):
        if file.endswith('.csv'):
            quantity_file += 1
            path_file_csv = os.path.join(path_folder_tuc, file)
            
            try:
                data = pd.read_csv(path_file_csv)
                data = data.to_numpy()
                if(data.shape[0] < step):
                    n = step // data.shape[0]

                    data = np.concatenate([data] * (n + 1))
                list_len_frame.append([data.shape[0], path_file_csv])
                data = data.reshape(-1, 40, 2)
                
                for i in range(step, data.shape[0]):
                    X.append(data[i - step: i, :])
                    Y.append(1)
            except:
                list_file_error.append(path_file_csv)
                
    # Data không tục
    for file in os.listdir(path_folder_khong_tuc):
        if file.endswith('.csv'):
            quantity_file += 1
            path_file_csv = os.path.join(path_folder_khong_tuc, file)
            
            try:
                data = pd.read_csv(path_file_csv)
                data = data.to_numpy()
                if(data.shape[0] < step):
                    n = step // data.shape[0]

                    data = np.concatenate([data] * (n + 1))

                list_len_frame.append([data.shape[0], path_file_csv])
                data = data.reshape(-1, 40, 2)
                
                for i in range(step, data.shape[0]):
                    X.append(data[i - step: i, :])
                    Y.append(0)
            except:
                list_file_error.append(path_file_csv)
    
    print('Đã lấy xong dữ liệu của: ' + folder)
    print('Tổng số file lỗi là: ' + str(len(list_file_error)) + '/' + str(quantity_file))
    print('File lỗi: ', 'không có' if list_file_error == [] else list_file_error)
    print('==============================================================================================================')
list_len_frame.sort(key= lambda x : x[0])
print('\t\t\t\t\t Đã lấy xong dữ liệu tất cả')
print('Min frame là: ', list_len_frame[0])
print('File min frame: ' + list_len_frame[0][1])
print(list_len_frame)

Đã lấy xong dữ liệu của: 01
Tổng số file lỗi là: 0/0
File lỗi:  không có
Đã lấy xong dữ liệu của: 02
Tổng số file lỗi là: 0/170
File lỗi:  không có
Đã lấy xong dữ liệu của: 03
Tổng số file lỗi là: 0/170
File lỗi:  không có
Đã lấy xong dữ liệu của: 04
Tổng số file lỗi là: 0/170
File lỗi:  không có
Đã lấy xong dữ liệu của: 05
Tổng số file lỗi là: 0/65
File lỗi:  không có
Đã lấy xong dữ liệu của: 06
Tổng số file lỗi là: 0/0
File lỗi:  không có
Đã lấy xong dữ liệu của: 07
Tổng số file lỗi là: 0/86
File lỗi:  không có
Đã lấy xong dữ liệu của: 08
Tổng số file lỗi là: 0/170
File lỗi:  không có
Đã lấy xong dữ liệu của: 09
Tổng số file lỗi là: 0/0
File lỗi:  không có
Đã lấy xong dữ liệu của: 10
Tổng số file lỗi là: 0/0
File lỗi:  không có
Đã lấy xong dữ liệu của: 11
Tổng số file lỗi là: 0/0
File lỗi:  không có
Đã lấy xong dữ liệu của: 12
Tổng số file lỗi là: 0/0
File lỗi:  không có
Đã lấy xong dữ liệu của: 13
Tổng số file lỗi là: 0/0
File lỗi:  không có
Đã lấy xong dữ liệu của: 14
Tổng số file 

Chuyển về numpy

In [18]:
X = np.array(X)
Y = np.array(Y)
Y = Y.reshape(-1, 1)
print(X.shape, Y.shape)

(101074, 32, 40, 2) (101074, 1)


# Phần này là training bằng tensorflow, muốn training bằng pytorch thì chuyển numpy sang tensor và training    

Chia dữ liệu

In [19]:
import pickle
pickle.dump(X, open('X.pkl', 'wb'))
pickle.dump(Y, open('Y.pkl', 'wb'))

In [20]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2)


Tạo model

In [26]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Reshape, Dense, LSTM, Dropout, BatchNormalization, Bidirectional, GlobalAveragePooling1D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import F1Score
# Tham số mô hình
TIME_STEPS = step  # Số bước thời gian (chuỗi đầu vào)
FEATURES = 80    # Số đặc trưng mỗi bước thời gian

# Input Layer
inputs = Input(shape=(32, 40, 2))
x = Reshape((inputs.shape[1], inputs.shape[2] * inputs.shape[3]))(inputs)

# LSTM Layer 1
x = Bidirectional(LSTM(64, activation='tanh', return_sequences=True, kernel_regularizer=l2(1e-4)))(x)

# LSTM Layer 2
x = Bidirectional(LSTM(128, activation='tanh', return_sequences=True, kernel_regularizer=l2(1e-4)))(x)
x = BatchNormalization()(x)
x = Dropout(0.15)(x)

# LSTM Layer 3
x = Bidirectional(LSTM(256, activation='tanh', return_sequences=True, kernel_regularizer=l2(1e-4)))(x)
x = BatchNormalization()(x)
x = Dropout(0.15)(x)


# GlobalAveragePooling1D để giảm chiều dữ liệu
x = GlobalAveragePooling1D()(x)
x = Dropout(0.2)(x)

# Dense Layer
x = Dense(16, activation="tanh", kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.2)(x)

# Output Layer (Binary Classification)
outputs = Dense(1, activation="sigmoid")(x)

# Tạo model
model = Model(inputs, outputs)

# Giảm Learning Rate (1e-4 hoặc 1e-5 nếu cần)
optimizer = Adam(learning_rate=1e-5)

# Compile model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall', F1Score()])

# In tóm tắt mô hình
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 32, 40, 2)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_7 (Reshape)             │ (None, 32, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 32, 128)        │        74,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 32, 256)        │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 32, 512)        │     1,050,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 512)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │         8,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,399,329 (5.34 MB)

 Trainable params: 1,397,793 (5.33 MB)

 Non-trainable params: 1,536 (6.00 KB)

Training

In [27]:
history = model.fit(xtrain, ytrain, batch_size= 16, epochs= 20, validation_data= (xtest, ytest))

Epoch 1/20
  64/5054 ━━━━━━━━━━━━━━━━━━━━ 7:07 86ms/step - accuracy: 0.5166 - f1_score: 0.4687 - loss: 0.8941 - precision: 0.3234 - recall: 0.5210

KeyboardInterrupt: 

Lưu model

In [ ]:
model.save('model_lstm.keras')

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    metrics = ['loss', 'accuracy', 'precision', 'recall', 'f1_score']
    titles = ['Loss', 'Accuracy', 'Precision', 'Recall', 'f1score']
    ylabel = ['Loss', 'Accuracy', 'Precision', 'Recall', 'f1score']

    plt.figure(figsize=(15, 12))
    for i, metric in enumerate(metrics):
        plt.subplot(3, 3, i + 1)
        plt.plot(history.history[metric], label=f'Train {metric}')
        plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric}')
        plt.xlabel('Epochs')
        plt.ylabel(ylabel[i])
        plt.legend()
        plt.title(f'{titles[i]} during training')

    plt.tight_layout()
    plt.show()
plot_training_history(history)